In [2]:
import torch
torch.__version__

'1.9.0'

In [18]:
import numpy as np
arr_1_d = np.asarray([1])
#print(arr_1_d)

arr_2_d = np.asarray([[1, 2], [3, 4]])
#print(arr_2_d)
#print(arr_2_d.reshape(1,4))

interest_score = np.random.randint(10, size=(4, 3))
#print(interest_score)
#print(np.sum(interest_score, axis=0))
#print(np.sum(interest_score, axis=1))

a = np.arange(18).reshape(3,2,3)
print(a)

print(a.max(axis=0))
print(a.max(axis=1))
print(a.max(axis=2))

[[[ 0  1  2]
  [ 3  4  5]]

 [[ 6  7  8]
  [ 9 10 11]]

 [[12 13 14]
  [15 16 17]]]
[[12 13 14]
 [15 16 17]]
[[ 3  4  5]
 [ 9 10 11]
 [15 16 17]]
[[ 2  5]
 [ 8 11]
 [14 17]]
